# Домашнее задание

Применить к прошлому домашнему заданию минимум по одному паттерну из каждой категории (порождающие, структурные, поведенческие). Код должен соответствовать SOLID принципам. Обосновать использование паттерна.

### Фабричный метод
Соревнования можно проводить разные, не только автогонки, скажем, лыжные гонки или соревнования по гребле. Тип участников соревнования будет меняться в зависимости от проводимого мероприятия
### Адаптер
Время прохождения дистанции зависит от параметров нескольких различных классов, поэтому я использую адаптер
### Итератор
Т.к. у меня разные типы участников соревнования и для подведения итогов важно знать результат каждого, то необходимо делать обход различных типов

In [17]:
#!/usr/bin/env python

from random import randint

class Competitor:
    started = False
    def start(self):
        self.started = True
    
    def get_penalty(self):
        raise NotImplementedError('subclasses must override get_penalty()!')
        
        
class Car(Competitor):
    def __init__(self, name, max_speed, drag_coef, time_to_max):
        self.name = name
        self.max_speed = max_speed
        self.drag_coef = drag_coef
        self.time_to_max = time_to_max
        
    def get_penalty(self):
        return 0
    
    
class Biathlete(Competitor):
    def __init__(self, name, max_speed, drag_coef, time_to_max, accuracy):
        self.name = name
        self.max_speed = max_speed
        self.drag_coef = drag_coef
        self.time_to_max = time_to_max
        self.accuracy = accuracy
        
    def get_penalty(self):
        return 20*(1 - self.accuracy)
    
    
class Weather:
    def __init__(self, wind_speed):
        self.wind_speed = wind_speed
        
        
class CompetitionParametrs(Competitor, Weather):
    def __init__(self, distance, competitor, weather):
        self.distance = distance
        self.name = competitor.name
        self.max_speed = competitor.max_speed
        self.drag_coef = competitor.drag_coef
        self.time_to_max = competitor.time_to_max
        self.wind_speed = weather.wind_speed
        self.penalty = competitor.get_penalty()
        self.started = competitor.started
        
    def computed_time(self):
        if not self.started:
            return "Didn't start"
        competitor_time = 0
        for distance in range(self.distance):
            _wind_speed = randint(0, self.wind_speed)
            if competitor_time == 0:
                _speed = 1
            else:
                _speed = (competitor_time / self.time_to_max) * self.max_speed
                if _speed > _wind_speed:
                    _speed -= (self.drag_coef * _wind_speed)
            competitor_time += float(1) / _speed
        return competitor_time + self.penalty
        
        
class CompetitorsIterator:
    _competitions = None
    _current_position = -1
    
    def __init__(self, competitions):
        self._competitions = competitions
        
    def get_next(self):
        if self.has_more():
            self._current_position += 1
            return self._competitions[self._current_position]
        
    def has_more(self):
        return self._current_position + 1 < len(self._competitions)
        

class Competition:
    def __init__(self, name, distance, competitors, weather):
        self.name = name
        self.distance = distance
        self.competitors = competitors
        self.weather = weather
        
    def start(self):
        print('---{}---'.format(self.name))
        iterator = None
        if self.name == "Auto racing" or self.name == "Biathlon":
            iterator = CompetitorsIterator(self.competitors)
        if iterator == None:
            print('Not found iterator')
            return
        while(iterator.has_more()):
            competitor = iterator.get_next()
            competitor.start()
            param = CompetitionParametrs(self.distance, competitor, self.weather)
            res = param.computed_time()
            print(competitor.name, round(res))
        
        
distance = 10000
weather = Weather(20)

name_auto_racing = "Auto racing"
cars = [Car('ferrary', 340, 0.324, 26),
        Car('bugatti', 407, 0.39, 32),
        Car('toyota', 180, 0.25, 40),
        Car('lada', 180, 0.32, 56),
        Car('sx4', 180, 0.33, 44)]
competition_auto_racing = Competition(name_auto_racing, distance, cars, weather)
competition_auto_racing.start()

name_biatlon = "Biathlon"
biatletes = [Biathlete('Simon Schempp', 20, 0.3, 120, 0.88),
             Biathlete('Emil Hegle Svendsen', 26, 0.33, 100, 0.88),
             Biathlete('Martin Fourcade', 26, 0.25, 140, 0.90),
             Biathlete('Anton Shipulin', 25, 0.29, 130, 0.88)]
competition_biatlon = Competition(name_biatlon, distance, biatletes, weather)
competition_biatlon.start()

---Auto racing---
ferrary 39
bugatti 40
toyota 67
lada 80
sx4 71
---Biathlon---
Simon Schempp 364
Emil Hegle Svendsen 291
Martin Fourcade 341
Anton Shipulin 337
